In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [0]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [0]:
def createmodel():
    model = Sequential()
    lstm_out=196
    embed_dim=128
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary())

In [7]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [8]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 19s - loss: 0.8326 - accuracy: 0.6419
Epoch 2/20
 - 16s - loss: 0.6831 - accuracy: 0.7100
Epoch 3/20
 - 16s - loss: 0.6188 - accuracy: 0.7386
Epoch 4/20
 - 17s - loss: 0.5758 - accuracy: 0.7608
Epoch 5/20
 - 16s - loss: 0.5372 - accuracy: 0.7802
Epoch 6/20
 - 17s - loss: 0.5037 - accuracy: 0.7946
Epoch 7/20
 - 17s - loss: 0.4628 - accuracy: 0.8137
Epoch 8/20
 - 16s - loss: 0.4292 - accuracy: 0.8271
Epoch 9/20
 - 17s - loss: 0.4022 - accuracy: 0.8336
Epoch 10/20
 - 16s - loss: 0.3829 - accuracy: 0.8435
Epoch 11/20
 - 16s - loss: 0.3602 - accuracy: 0.8548
Epoch 12/20
 - 16s - loss: 0.3390 - accuracy: 0.8609
Epoch 13/20
 - 17s - loss: 0.3233 - accuracy: 0.8689
Epoch 14/20
 - 18s - loss: 0.3061 - accuracy: 0.8774
Epoch 15/20
 - 16s - loss: 0.2945 - accuracy: 0.8812
Epoch 16/20
 - 17s - loss: 0.2702 - accuracy: 0.8892
Epoch 17/20
 - 17s - loss: 0.2730 - accuracy: 0.8915
Epoch 18/20
 - 17s - loss: 0.2594 - accuracy: 0.8969
Epoch 19/20
 - 17s - loss: 0.2468 - accuracy: 0.8999
Ep

In [0]:
model.save('Nikhitha.h5')

In [10]:
from keras.models import load_model
model= load_model('Nikhitha.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

2
Positive


2. Apply GridSearchCV on the source code provided in the class

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 43s - loss: 0.8326 - accuracy: 0.6431
Epoch 1/1
 - 43s - loss: 0.8245 - accuracy: 0.6435
Epoch 1/1
 - 43s - loss: 0.8277 - accuracy: 0.6426
Epoch 1/1
 - 43s - loss: 0.8315 - accuracy: 0.6433
Epoch 1/1
 - 42s - loss: 0.8277 - accuracy: 0.6441
Epoch 1/2
 - 40s - loss: 0.8282 - accuracy: 0.6434
Epoch 2/2
 - 40s - loss: 0.6889 - accuracy: 0.7074
Epoch 1/2
 - 43s - loss: 0.8290 - accuracy: 0.6422
Epoch 2/2
 - 43s - loss: 0.6887 - accuracy: 0.7131
Epoch 1/2
 - 40s - loss: 0.8282 - accuracy: 0.6450
Epoch 2/2
 - 42s - loss: 0.6808 - accuracy: 0.7143
Epoch 1/2
 - 43s - loss: 0.8363 - accuracy: 0.6421
Epoch 2/2
 - 42s - loss: 0.6882 - accuracy: 0.7049
Epoch 1/2
 - 42s - loss: 0.8213 - accuracy: 0.6464
Epoch 2/2
 - 43s - loss: 0.6709 - accuracy: 0.7141
Epoch 1/1
 - 21s - loss: 0.8419 - accuracy: 0.6395
Epoch 1/1
 - 20s - loss: 0.8355 - accuracy: 0.6439
Epoch 1/1
 - 21s - loss: 0.8375 - accuracy: 0.6376
Epoch 1/1
 - 21s - loss: 0.8412 - accuracy: 0.6351
Epoch 1/1
 - 21s - loss: 0.8328

3. Apply the code on spamdata set availablein thesourcecode (text classification on the spam.csvdata set)

In [0]:
data1 = pd.read_csv('/content/spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]

In [0]:
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [22]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 34s - loss: 0.1557 - accuracy: 0.9437
Epoch 2/5
 - 33s - loss: 0.0423 - accuracy: 0.9871
Epoch 3/5
 - 33s - loss: 0.0208 - accuracy: 0.9944
Epoch 4/5
 - 32s - loss: 0.0084 - accuracy: 0.9979
Epoch 5/5
 - 31s - loss: 0.0061 - accuracy: 0.9981
0.09851680500756964
0.9815117120742798
['loss', 'accuracy']
